In [111]:
# -*- coding:utf-8 -*-
import pymongo
import requests
import multiprocessing
from bs4 import BeautifulSoup
import re
from time import sleep
import datetime
import random

def get_monggo_db():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn

def get_mongo_crawl_coll():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn['crawl_58_tianhe_shangpu']

def clean_mongo_crawl_coll():
    db = get_monggo_db()
    re = db.drop_collection("crawl_58_tianhe_shangpu")
    print(re)    

def get_type_from_description(desc):
    xs = split(desc)
    for x in xs:
        print(x)
    
    
def rinse_description(name, area, desc):
    desc = desc.replace(name, '')
    rslt = desc.replace(area, '')
    return rslt.strip()
    
# 请求天河商铺列表和基本信息
def req_tianhe_sp_list():
    base_url = 'http://gz.58.com/tianhe/shangpucz/pn'
    coll = get_mongo_crawl_coll()
    for page_num in range(1, 71):
        page_url = '%s%d' % (base_url, page_num)
        html_doc = requests.get(page_url).content
        soup = BeautifulSoup(html_doc, 'html.parser')
        trs = soup.find("table", class_="tbimg").findAll('tr')
        for tr in trs:
            td2 = tr.find('td', class_='t')
            url = td2.find('a', class_='t').get('href')
            name = td2.find('a', class_='t').text
            area = td2.find('span', class_='ico area').find('a').text
            description = td2.text
            description = rinse_description(name, area, description)
            last2tds = tr.findAll('td', class_='tc')
            td3 = last2tds[0]
            price = td3.text
            td4 = last2tds[1]
            centiares = td4.find('b').text
            doc = {                
                'name': name,
                'url': url
            }
            coll.update_one({'url': url}, {'$set': doc}, True)   

def req_tianhe_sp():
    coll = get_mongo_crawl_coll()
    cursor = coll.find()
    for cur in cursor: 
        html_doc = requests.get(cur.get('url')).content
        soup = BeautifulSoup(html_doc, 'html.parser')
        lis = soup.find('ul', class_='info').findAll('li')            
        desc = ''
        for li in lis:
            desc += li.text        
        cur['description'] = desc
        coll.save(cur)   

if __name__ == '__main__':
    print('start crawling')
    clean_mongo_crawl_coll()    
    req_tianhe_sp_list()
    req_tianhe_sp()
    print('ended ended')

start crawling


/home/suncc/.pyenv/versions/3.6.1/envs/env361/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead


ended ended
